In [1]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd "/content/drive/My Drive/L_M_word_based"

/content/drive/My Drive/L_M_word_based


In [0]:
with open("sample1.txt", "r") as file:
  data = file.read()

In [0]:
# with open("persian", "r") as file:
#   stop_words = file.read()

In [0]:
import re
def cleaning_data(data):
    data = re.sub("[\\u202c]", "", data)
    data = re.sub("[\\u202b]", "", data)
    data = re.sub("[\\u202a]", "", data)
    data = re.sub("[\\ufeff]", "", data)
    data = re.sub("[\\u200d]", "", data)
    data = re.sub("[\\uf089]", "", data)
    data = re.sub("[\\u200f]", "", data)
    data = re.sub("[\\u200b]", "", data)
    data = re.sub("[\\uf08b]", "", data)
    data = re.sub("[\\uf08c]", "", data)
    data = re.sub("[\d]", " ", data)
    data = re.sub("[\s]", " ", data)
    data = re.sub('[/(){}\[\]\|@,;!٪×،*ـ+؟؛"" ... .. . <> _ - :]', " ", data)
    data = re.sub('[!٬٫﷼٪×*)(ـ+}|؛؟<>‌ ÷؛«» "" - �]', " ", data)
    return data.split()

In [6]:
tokens = cleaning_data(data)
len(tokens)

138666

In [0]:
# stop_words = cleaning_data(stop_words)
# stop_words = stop_words.split(" ")
# clean_data = [w for w in data.split() if not w in stop_words if len(w)>2]

In [8]:
print("total tokens: {}".format(len(tokens)))
print("unique tokens: {}".format(len(set(tokens))))

total tokens: 138666
unique tokens: 13514


In [0]:
train_len = 25+1 # 50 training words , then one target word

# Empty list of sequences
text_sequences = []

for i in range(train_len, len(tokens)):
    
    # Grab train_len# amount of characters
    seq = tokens[i-train_len:i]
    
    # Add to list of sequences
    text_sequences.append(seq)

In [10]:
len(text_sequences)

138640

In [11]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)

In [13]:
vocabulary_size = len(tokenizer.word_counts)
print("Vocabulary size: ", vocabulary_size)

Vocabulary size:  13514


In [0]:
# sequences = []
# for i in range(1, vocab_size):
#     sequence = encoded[i-1: i+1]
#     sequences.append(sequence)
# print("Total sequences: ", len(sequences))

In [0]:
from keras.utils import to_categorical, plot_model
import numpy as np
sequences = np.array(sequences)

In [0]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense, LSTM
from keras.callbacks import ModelCheckpoint

In [0]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    plot_model(model, to_file="model.png")
    
    return model

In [0]:
from keras.utils import to_categorical

In [0]:
X = sequences[:,:-1]

In [0]:
y = sequences[:,-1]

In [0]:
y = to_categorical(y, num_classes=vocabulary_size+1)

In [0]:
seq_len = X.shape[1]

In [23]:
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            337875    
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 150)           105600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense_1 (Dense)              (None, 150)               22650     
_________________________________________________________________
dense_2 (Dense)              (None, 13515)             2040765   
Total params: 2,687,490
Trainable params: 2,687,490
Non-trainable params: 0
_________________________________________________________________


In [0]:
from pickle import dump,load

In [0]:
# mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
# model.fit(X, y, batch_size=128, epochs=100,verbose=1, callbacks=[mc])

In [0]:
from keras.models import load_model
# model.save("model1.h5")

In [27]:
# model01 = load_model("model1.h5")

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [28]:
checkpoint = ModelCheckpoint("model01.h5", monitor = 'loss', save_best_only = True, mode = 'min') 
model01.fit(X, y, batch_size=128, epochs=100,verbose=1, callbacks=[checkpoint])

Epoch 1/100
138640/138640 [==============================] - 95s 683us/step - loss: 0.9606 - accuracy: 0.7451
Epoch 2/100
138640/138640 [==============================] - 93s 671us/step - loss: 0.9322 - accuracy: 0.7529
Epoch 3/100
138640/138640 [==============================] - 93s 667us/step - loss: 0.9430 - accuracy: 0.7491
Epoch 4/100
138640/138640 [==============================] - 93s 668us/step - loss: 0.9389 - accuracy: 0.7505
Epoch 5/100
138640/138640 [==============================] - 93s 671us/step - loss: 0.9349 - accuracy: 0.7516
Epoch 6/100
138640/138640 [==============================] - 92s 666us/step - loss: 0.9268 - accuracy: 0.7540
Epoch 7/100
138640/138640 [==============================] - 92s 664us/step - loss: 0.9205 - accuracy: 0.7551
Epoch 8/100
138640/138640 [==============================] - 93s 672us/step - loss: 0.9127 - accuracy: 0.7569
Epoch 9/100
138640/138640 [==============================] - 92s 662us/step - loss: 0.9272 - accuracy: 0.7509
Epoch 10/1

In [0]:
model01.save("model_final.h5")

In [31]:
final_model = load_model("final_model.h5")

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [0]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict Class Probabilities for each word
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

In [42]:
text_sequences[0]

['خطبهها',
 'خطبه',
 'آفرینش',
 'آسامن',
 'و',
 'زمین',
 'امام',
 'در',
 'این',
 'سخرنانی',
 'از',
 'آفرینش',
 'آسامن',
 'و',
 'زمین',
 'و',
 'آفرینش',
 'انسان',
 'یاد',
 'م',
 'ی',
 'كند',
 'سپاس',
 'خداوندی',
 'را',
 'كه']

In [0]:
import random
random.seed(101)
random_pick = random.randint(0,len(text_sequences))

In [0]:
random_seed_text = text_sequences[random_pick]

In [46]:
random_seed_text

['ایامن',
 'میتوان',
 'دسرتسی',
 'پیداكرد',
 'با',
 'ایامن',
 'علم',
 'و',
 'دانش',
 'آبادان',
 'است',
 'و',
 'با',
 'آگاهی',
 'الزم',
 'انسان',
 'از',
 'مرگ',
 'میهراسد',
 'و',
 'با',
 'مرگ',
 'دنیا',
 'پایان',
 'میپذیرد',
 'و']

In [0]:
seed_text = ' '.join(random_seed_text)

In [60]:
seed_text

'ایامن میتوان دسرتسی پیداكرد با ایامن علم و دانش آبادان است و با آگاهی الزم انسان از مرگ میهراسد و با مرگ دنیا پایان میپذیرد و'

In [50]:
generate_text(final_model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=10)

'از رفت و آمد او پراكنده شود آنها پر نامه'

In [0]:
def read_file(filepath):
    
    with open(filepath) as f:
        str_text = f.read()
    
    return str_text

In [0]:
full_text = read_file('sample1.txt')

In [59]:
for i,word in enumerate(full_text.split()):
    if word == 'زمین':
        print(' '.join(full_text.split()[i-5:i+5]))
        print('\n')

سخرنانی از آفرینش آسامن و زمین و آفرینش انسان یاد


وسیلۀ كوه‏ها اضطراب و لرزش زمین را به آرامش تبـدیل‬


و هوایِ به آسامن و زمین راه یافته را آفرید‪


آنها پاهایشان در طبقات پایین زمین قرار داشته و گردن​هاشان‬


بر فراز انسان‏ها‪،‬‬ ‫گاهوارۀ گسرتدۀ زمین در زیر پای آنها‪


‫در روزگاری كه مردم روی زمین دارای مذاهب پراكنده‪ ،‬خواسته‏های‬


خواهان رسكشی و فساد در زمین نباشند‬ ‫‪11‬‬ ‫خطبه‏ها (


راهنامیی نداشتید‪ .‬تشنه كام‬ ‫هرچه زمین را می‏كندید قطرۀ آبی


و برتری جویی را بر زمین نهید‪.‬‬ ‫رستگار شد آن


خدا عاریت ده‪ ،‬پای بر زمین میخكوب‬ ‫كن‪،‬‬ ‫به صفوف


را بلند كرده‪ ،‬سوار رابر زمین می‏كوبد‪ ،‬و با دام​های


‫بهره هر كدام شام از زمین به اندازه طول و


گنجینه‏های آیین خدا و اركان زمین است‪ .‬خود را به


پر پیچ و خم‪ ،‬نه زمین گسرتده و نه آفریده‏های


آنها استفاده كرد و روانۀ زمین كرده‪ ،‬آنها‬ ‫را از


گروهی‬ ‫دیگر‪ ،‬قدمهایشان تا ژرفای زمین پایین رفته و چونان


و‬ ‫ُسستی‪ ،‬آن را بر زمین منی‏گذارند‪ .‬در متام آسامنها


،‬قسمت​های رسكش آب از سنگینی زمین فرو‬ ‫نشست و هیجانِ


بر اثر

'\x0c\u202bخطبه\u200fها ( \u202a) 1\u202c\u202c\n\u202bخطبه \u202a1\u202c\u202c\n\u202bآفرینش آسامن و زمین\u202c\n\n\u202b(امام \u202a \uf089\u202cدر این سخرنانی از آفرینش آسامن و زمین و آفرینش انسان یاد م \u200fی كند\u202a).\u202c\u202c\n\u202bسپاس خداوندی را كه سخنوران از ستودن او عاجزند و حسابگران\u202c\n\u202bاز شامرش نعمتهای او ناتوان و تالشگران از ادای حق او درمانده\u200bاند\u202a.\u202c\u202c\n\u202bخدایی كه افكار ژرف اندیش\u202a ،\u202cذات او را درك منی\u200fكنند و دسـت غ ّواصان\u202c\n\u202bدریای علوم به او نخواهد رسیــد\u202a .\u202cپروردگـاری كه برای صفات او ح ّد\u202c\n\u202bو مرزی وجود ندارد و تعریف كاملی منی\u200fتوان یافت و برای خدا وقتی\u202c\n\u202bمع ّین و رسآمدی مشخّص منی\u200fتوان تعیین كرد\u202a.\u202c\u202c\n\u202bمخلوقات را با قدرت خود آفرید\u202a ،\u202cو با رحمت خود بادهـا را به حركـت\u202c\n\u202bدرآورد و به وسیلۀ كوه\u200fها اضطراب و لرزش زمین را به آرامش تبـدیل\u202c\n\u202bكرد\u202a.\u202c\u202c\n\u202bرسآغاز دین\u202a ،\u202cخداشناسی است؛ و كامل شناخت خدا\